# Example for negative times values (The example file is a [testfile of pyglotaran](https://github.com/glotaran/pyglotaran/blob/master/glotaran/builtin/file_formats/sdt/test/data_files/temporal.sdt))

In [1]:
import numpy as np
from sdtfile import SdtFile, FileRevision

In [2]:
file = SdtFile("temporal.sdt")

In [3]:
file.times

[array([-0.00000000e+00, -1.22179723e-11, -2.44359446e-11, ...,
        -5.00081606e-08, -5.00203786e-08, -5.00325965e-08])]

## Since negative times aren't reasonable, closer investigation is needed

In [4]:
FileRevision(file.header.revision).revision

15

In [5]:
file.measure_info

[rec.array([(b'11:44:42', b'2017-04-27', b'3D0004', 0, -25.490196, 80., -30.236221, 5., -9.826772, 1, 4., 5.0044815e-08, 15, 50.64713, 3.9215686, 95.68627, 4096, 0, 1, 1, 1, 300., 100000., 0, 78, 0, 1, 0., 128, 1, 0, b'SPC-830', -29.411764, 1, 1, 1, 1, 2, 0, 1, 1, 1, 6, 2.e-07, 1, 16, 0, 0, 0, 0, 0, 4, 830, 260, 0., 0, 1, (264, 32768, 186.80542, 1, 1, 1, 20100096., 5504., 2880., 2560., 20100352., 6080., 3168., 2848., 0, 0.), (0, 0, 0, 0., 0, 0, 0., 0, 0, 0, 0, 0, 0), 0, 0, 0, 0, 0, 0, 0, 0, 0, (0., 0., 0, 0, 0., 0))],
           dtype=[('time', 'S9'), ('date', 'S11'), ('mod_ser_no', 'S16'), ('meas_mode', '<i2'), ('cfd_ll', '<f4'), ('cfd_lh', '<f4'), ('cfd_zc', '<f4'), ('cfd_hf', '<f4'), ('syn_zc', '<f4'), ('syn_fd', '<i2'), ('syn_hf', '<f4'), ('tac_r', '<f4'), ('tac_g', '<i2'), ('tac_of', '<f4'), ('tac_ll', '<f4'), ('tac_lh', '<f4'), ('adc_re', '<i2'), ('eal_de', '<i2'), ('ncx', '<i2'), ('ncy', '<i2'), ('page', '<u2'), ('col_t', '<f4'), ('rep_t', '<f4'), ('stopt', '<i2'), ('overfl', 'u

In [6]:
file.block_headers[0]

(0, 0, 18123, 26315, 3, 0, 1, 8192)

## Let's have a closer look at the [calculation of the times values](https://github.com/cgohlke/sdtfile/blob/e085fa55429988ccf2a0671c2fa6ba4a45c18b39/sdtfile/sdtfile.py#L276)

In [7]:
tac_r = file.measure_info[0].tac_r
tac_r

array([5.0044815e-08], dtype=float32)

In [8]:
tac_g = file.measure_info[0].tac_g
tac_g

array([15], dtype=int16)

In [9]:
adc_re = int(file.measure_info[0].adc_re)
adc_re

4096

### Original code for the multiplicative factor 

In [10]:
tac_r / float(tac_g*adc_re)

array([-1.2217972e-11], dtype=float32)

### The difference needs to be caused by `float(tac_g*adc_re)`

In [11]:
float(tac_g*adc_re)

-4096.0

In [12]:
float(tac_g[0]*adc_re)

61440.0

### The cause of the problem is a datatype overflow since mi.tac_g is int

In [13]:
np.array([15, 16, 17], dtype=np.int16) * adc_re

array([-4096,     0,  4096], dtype=int16)

## Possible solutions

In [14]:
tac_r / (float(tac_g)*adc_re)

array([8.145315e-13], dtype=float32)

In [15]:
tac_r[0] / float(tac_g[0]*adc_re)

8.145314862410362e-13